# Init


In [1]:
# Install necessary requirements
%pip install -r ../requirements.txt

# Change working directory to root
import os
if os.getcwd().endswith("notebooks"):
    %cd ..
    print(os.getcwd())

# Automatically reload changes in code
%load_ext autoreload
%autoreload 2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
/Users/dorin/Desktop/Tabular Data Science/Association-Rules-for-Concept-Drifting
/Users/dorin/Desktop/Tabular Data Science/Association-Rules-for-Concept-Drifting


In [2]:
import logging
from typing import List

import pandas as pd

from association_finder.concept_drifts_finder import ConceptDriftsFinder
from association_finder.models import Transaction, ConceptDriftResult
logging.basicConfig()

# Read and parse file

In [17]:
from sklearn.model_selection import train_test_split
from typing import Dict, Tuple, Optional
from dataclasses import dataclass
import numpy as np

np.random.seed(0)

train_dataset_path = "datasets/houseprices/train.csv"
df = pd.read_csv(train_dataset_path, index_col='Id')
df_train, df_val = train_test_split(df, test_size=0.3)

@dataclass
class TrainParams:
    
    na_columns_mean: Dict[int, float]
    numerical_columns_cut: Dict[str, List[float]]
    
def preprocess(df, train_params: Optional[TrainParams] = None):
    
    # Defining numeric and categorical columns
    numeric_columns = df.dtypes[(df.dtypes == "float64") | (df.dtypes == "int64")].index.tolist()
    very_numerical = [nc for nc in numeric_columns if df[nc].nunique() > 20]
    categorical_columns = [c for c in df.columns if c not in numeric_columns]
    ordinals = list(set(numeric_columns) - set(very_numerical))

    # Filling Null Values with the column's mean
    na_columns = df[very_numerical].isna().sum()
    na_columns = na_columns[na_columns > 0]

    na_columns_mean = {}
        
    for nc in na_columns.index:
        if train_params is None:
            column_mean = df[nc].mean()

            # Save mean
            na_columns_mean[nc] = column_mean
        else:
            column_mean = train_params.na_columns_mean[nc]

        df[nc].fillna(column_mean, inplace=True)


    # Dropping and filling NA values for categorical columns:
    # drop if at least 70% are NA:
    nul_cols = df[categorical_columns].isna().sum() / len(df)
    drop_us = nul_cols[nul_cols > 0.7]
    df = df.drop(drop_us.index, axis=1)

    # Fill with a new 'na' category:
    categorical_columns = list(set(categorical_columns) - set(drop_us.index))
    df[categorical_columns] = df[categorical_columns].fillna('na')

    # Fill Null values in ordinals with a new '-1' ordinal:
    df[ordinals] = df[ordinals].fillna(-1)
    
    df = df.copy()

    # Bin numerical data
    numerical_columns_cut = {}
    for c in very_numerical:
        if train_params is None:
            try:
                # df[c] = pd.qcut(df[c], 5, labels=["very low", "low", "medium", "high", "very high"])
                df[c], bins = pd.qcut(df[c], 5, labels=[1, 2, 3, 4, 5], retbins=True)
            except:
                # sometimes for highly skewed data, we cannot perform qcut as most quantiles are equal
                # df[c] = pd.cut(df[c], 5, labels=["very low", "low", "medium", "high", "very high"])
                df[c], bins = pd.cut(df[c], 5, labels=[1, 2, 3, 4, 5], retbins=True)

            # Make bin edges larger (infinity and -infinity)
            bins = np.concatenate(([-np.inf], bins[1:-1], [np.inf]))

            # Save bin
            numerical_columns_cut[c] = bins
            
        else:
            # Use existing train bins
            bins = train_params.numerical_columns_cut[c]
            df[c] = pd.cut(df[c], labels=[1, 2, 3, 4, 5], bins=bins)
        
    return df, TrainParams(na_columns_mean, numerical_columns_cut),[d for d in drop_us.keys()]

df_train_prep, train_params, dropped_columns = preprocess(df_train)
print(dropped_columns)

# Focusing on prominent columns:
good_columns = [column for column in ['OverallQual', 'YearBuilt', 'YearRemodAdd', 'OverallCond', 'BldgType', 'LotArea',
                'GrLivArea', 'FullBath', 'BedroomAbvGr', 'LotFrontage', 'TotalBsmtSF', 'SalePrice'] if column not in 
                dropped_columns]
# TODO: Check where Amit took the good columns list from

target_column = "SalePrice"

one_hot_columns = [column for column in ['BldgType'] if column not in dropped_columns]

def split_X_y(df_prep: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    X_columns = list(set(good_columns) set(dropped_columns) - {target_column})
    X = df_prep[X_columns]

    for one_hot_column in one_hot_columns:
        X = pd.concat([X, pd.get_dummies(X[one_hot_column], prefix=one_hot_column)], axis=1)
        X = X.drop(columns=[one_hot_column])

    y = df_prep[target_column]
    
    return X, y

['Alley', 'PoolQC', 'Fence', 'MiscFeature']


# Baseline

In [12]:
from sklearn.linear_model import LogisticRegression
from typing import Tuple

X_train, y_train = split_X_y(df_train_prep)
X_val, y_val = split_X_y(preprocess(df_val, train_params)[0])

clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/Users/dorin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
print(f"Train accuracy: {clf.score(X_train, y_train)}")
print(f"Validation accuracy: {clf.score(X_val, y_val)}")

Train accuracy: 0.6653620352250489
Validation accuracy: 0.6164383561643836


# Build model using rules

In [14]:
from association_finder.concept_engineering import ConceptEngineering

concept_engineering = ConceptEngineering()
X_train_rules = concept_engineering.fit_transform(X_train, df_train_prep[good_columns], target_column, one_hot_columns)
X_val_rules = concept_engineering.transform(X_val)

clf_rules = LogisticRegression(random_state=0).fit(X_train_rules, y_train)

print(f"Train accuracy: {clf_rules.score(X_train_rules, y_train)}")
print(f"Validation accuracy: {clf_rules.score(X_val_rules, y_val)}")

#Train accuracy: 0.662426614481409
# Validation accuracy: 0.6415525114155252

Train accuracy: 0.662426614481409
Validation accuracy: 0.6415525114155252


/Users/dorin/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Analyze concepts

In [10]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
concept_engineering.concepts_df

,left_hand_side,right_hand_side,confidence_before,confidence_after,support_before,support_after,concept_cutoff,concept_column
0,{'BldgType': '1Fam'},{'SalePrice': 1},1.000000,0.185229,1.000000,0.155359,2.8,OverallQual
1,{'FullBath': 1},{'SalePrice': 1},1.000000,0.374718,0.600000,0.163225,2.8,OverallQual
2,{'GrLivArea': 1},{'SalePrice': 1},1.000000,0.530000,1.000000,0.104228,2.8,OverallQual
3,{'YearBuilt': 1},{'SalePrice': 1},1.000000,0.509615,0.800000,0.104228,2.8,OverallQual
4,"{'BldgType': '1Fam', 'FullBath': 1}",{'SalePrice': 1},1.000000,0.361809,0.600000,0.141593,2.8,OverallQual
5,"{'FullBath': 1, 'GrLivArea': 1}",{'SalePrice': 1},1.000000,0.548387,0.600000,0.100295,2.8,OverallQual
6,{'BldgType': '1Fam'},{'SalePrice': 1},0.675325,0.142125,0.541667,0.119870,4.6,OverallQual
7,{'FullBath': 1},{'SalePrice': 1},0.739130,0.312997,0.531250,0.127430,4.6,OverallQual
8,"{'BldgType': '1Fam', 'FullBath': 1}",{'SalePrice': 1},0.721311,0.302941,0.458333,0.111231,4.6,OverallQual
9,{'BldgType': '1Fam'},{'SalePrice': 5},0.193470,0.967742,0.161943,0.882353,8.2,OverallQual
